In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
import pickle


In [15]:
data = pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

le_gender = LabelEncoder()
data['Gender'] = le_gender.fit_transform(data['Gender']) 

ohe_geo = OneHotEncoder()
geography_encoded = ohe_geo.fit_transform(data[['Geography']])
geography_encoded_df = pd.DataFrame(geography_encoded.toarray(), columns=ohe_geo.get_feature_names_out(['Geography']))
geography_encoded_df

data = pd.concat([data, geography_encoded_df], axis=1)
data = data.drop(['Geography'], axis=1)

with open('le_gender.pkl', 'wb') as file:
    pickle.dump(le_gender,file)
with open('ohe_geo.pkl', 'wb') as file:
    pickle.dump(ohe_geo,file) 

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save the scaler
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [16]:
# Define the model creation function and try different params(KerasClassifier)

def create_model(neuron=32, layers=1):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))
    model.add(Dense(neuron, activation='relu'))

    for _ in range(layers - 1):
        model.add(Dense(neuron, activation='relu'))
        
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [17]:
# create Keras classifier
model = KerasClassifier(layers = 1, neuron = 32, model=create_model,epochs=50, batch_size=10, verbose=0)

In [18]:
# Define Grid search Params
param_grid = {
    'neuron': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [19]:
grid = GridSearchCV(estimator=model,param_grid=param_grid, cv=3, n_jobs=1)
grid_result = grid.fit(X_train, Y_train)

#Printing Best suited one
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

KeyboardInterrupt: 